#Content Based filtering(Recommended Systems)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

#Create data frames

In [ ]:
users = ['Akash','Xajeel','Rayan','Ahmad Shah']
Movies = ['Money Hiest','Hells paradise','Death note','Berlin','Naroto']
Features = ['Action','Romantic','Comedy','Animie','Drama']

num_users = len(users)
num_movie = len(Movies)
num_features = len(Features)
num_recommendation = 2

#(1)How much the user i give rating movie j
#(2)One hot encoding of all the features of a movie

In [ ]:
user_movie = tf.constant([
                [4,5,3,0,8],
                [0,0,0,6,0],
                [3,3,4,5,7],
                [0,3,7,9,3]],dtype=tf.int32)

movie_feat = tf.constant([
                          [1,1,0,0,1],
                          [1,1,0,0,0],
                          [0,0,1,1,0],
                          [1,0,1,1,0],
                          [0,0,1,0,1]],dtype=tf.int32)

#Matrix multiplication to check how muuch a user i like the features of movie j

In [ ]:
user_features = tf.matmul(user_movie, movie_feat)
user_features = user_features / tf.reduce_sum(user_features, axis=1, keepdims=True)

In [ ]:
user_features

<tf.Tensor: shape=(4, 5), dtype=float64, numpy=
array([[0.20454545, 0.20454545, 0.25      , 0.06818182, 0.27272727],
       [0.33333333, 0.        , 0.33333333, 0.33333333, 0.        ],
       [0.21153846, 0.11538462, 0.30769231, 0.17307692, 0.19230769],
       [0.22641509, 0.05660377, 0.35849057, 0.30188679, 0.05660377]])>

#Selecting the top_features that user likes the most

In [ ]:
k = num_recommendation
Top_user_features = tf.math.top_k(user_features, k)[1]
Top_user_features

<tf.Tensor: shape=(4, 2), dtype=int32, numpy=
array([[4, 2],
       [0, 2],
       [2, 0],
       [2, 3]], dtype=int32)>

# Retrieving the top features name

In [ ]:
for i in range(num_users):
  features_name = [Features[int(index)] for index in Top_user_features[i]]
  print(f"Top features for {users[i]}: {features_name}")

Top features for Akash: ['Drama', 'Comedy']
Top features for Xajeel: ['Action', 'Comedy']
Top features for Rayan: ['Comedy', 'Action']
Top features for Ahmad Shah: ['Comedy', 'Animie']


# user_rating represents the predicted rating of a user for a particular movie.

In [ ]:
user_rating = tf.matmul(user_features, tf.transpose(tf.cast(movie_feat,tf.float64)))
user_rating

<tf.Tensor: shape=(4, 5), dtype=float64, numpy=
array([[0.68181818, 0.40909091, 0.31818182, 0.52272727, 0.52272727],
       [0.33333333, 0.33333333, 0.66666667, 1.        , 0.33333333],
       [0.51923077, 0.32692308, 0.48076923, 0.69230769, 0.5       ],
       [0.33962264, 0.28301887, 0.66037736, 0.88679245, 0.41509434]])>

In [ ]:
users_ratings_new = tf.where(
    tf.equal(user_movie, tf.zeros_like(user_movie)),
    tf.cast(user_rating, tf.float32),
    tf.zeros_like(tf.cast(user_movie, tf.float32))
)

#is part of the process to recommend top-rated movies to users based on the predicted ratings (users_ratings_new).

In [ ]:
top_movies = tf.nn.top_k(users_ratings_new, num_recommendation)[1]
top_movies

<tf.Tensor: shape=(4, 2), dtype=int32, numpy=
array([[3, 0],
       [2, 0],
       [0, 1],
       [0, 1]], dtype=int32)>

In [ ]:
for i in range(num_users):
 movie_names = [Movies [index] for index in top_movies[i]]
 print('{}: {}'.format(users[i],movie_names))

Akash: ['Berlin', 'Money Hiest']
Xajeel: ['Death note', 'Money Hiest']
Rayan: ['Money Hiest', 'Hells paradise']
Ahmad Shah: ['Money Hiest', 'Hells paradise']
